In [1]:
import numpy as np

from weight import *
from utils import save, load
from library import *
from process_library_terms import *

In [2]:
%%prun # profiling

### load data
# vicsek:
#arrays.npy - normal. arrays2.npy - very few particles. arrays3.npy - high density. arrays4.npy - high density, low noise.
#arrays5.npy - small domain, one clump
#datasets = ['arrays.npy', 'arrays2.npy', 'arrays3.npy']#, 'arrays5.npy']
#datasets = ['data_bu.npy'] # burgers' equation data
## it may be interesting to reintroduce regression on multiple datasets,
## but for now we're just not going to do it

#dataset = 'data_bu.npy'
import scipy.io
dataset = 'gas_1000_4_0.mat'
matdata = scipy.io.loadmat(dataset)
positions = matdata['pos_data']
vs = matdata['vs_data']
deltat = matdata['dt'][0][0]
dims = matdata['dims'][0]
#for i, dset in enumerate(datasets):
#positions, vs, deltat, dims = load(dataset, 4)
world_size = np.array([dims[0], dims[0], dims[1]])
#world_size[2] = world_size[2]-200 # start at frame 200
#print(positions.shape)
#positions = positions[:, :, 200:]
#vs = vs[:, :, 200:]
print(world_size)
#world_size[2] = 50 # restrict to first 50 time steps
data_dict = {}
data_dict['v'] = vs
v_obs = Observable('v', 1)
observables = [v_obs]

# fix random seed
np.random.seed(1)

# initial setup of dataset
srd = SRDataset(world_size, data_dict, positions, observables, 
                kernel_sigma=0.1, cg_res=20, deltat=deltat, cutoff=6) #kernel_sigma=0.6

# initialize libraries, domains, and weights
#srd.make_libraries(max_complexity=4, max_observables=3)
srd.make_libraries(max_complexity=6, max_observables=4)

dom_width = 1.5 #10 #80
dom_time = 50 #20
srd.make_domains(ndomains=30, domain_size=[dom_width, dom_width, dom_time], pad=0)
#srd.make_domains(ndomains=1, domain_size=[dom_width, dom_width, dom_time], pad=0)
srd.make_weights(m=16, qmax=1)
srd.make_library_matrices()

save('Q_bu.npy', srd.dxs, srd.libs)

[  6.28318531   6.28318531 750.        ]
 

         463604218 function calls (455328566 primitive calls) in 1641.911 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   643680  222.311    0.000  223.166    0.000 diff.py:478(_apply_to_array)
      450  178.874    0.397  809.882    1.800 process_library_terms.py:186(eval_cgp)
  7919460  171.156    0.000  417.561    0.000 convolution.py:15(gauss1d)
   675360  161.771    0.000  163.661    0.000 polynomial.py:664(polyval)
        1  107.032  107.032 1641.737 1641.737 process_library_terms.py:227(make_library_matrices)
 17699953   87.935    0.000   87.935    0.000 {method 'reduce' of 'numpy.ufunc' objects}
  7919460   64.575    0.000   64.575    0.000 {method 'outer' of 'numpy.ufunc' objects}
30155692/26869833   56.998    0.000  419.097    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
 11815460   55.461    0.000   55.461    0.000 {built-in method numpy.array}
 16571948   42.276    0.000  147

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2)= plt.subplots(ncols=2, figsize=(12,6))
print(srd.cgps)
print(srd.domains[2])
cg_arr = srd.cg_dict[srd.cgps[1], (0,), srd.domains[2]]
ax1.imshow(cg_arr[:, :, 0], origin='lower', extent=[133, 172, 203, 242])
ax1.scatter(srd.scaled_pts[:, 1, 279].flatten(), srd.scaled_pts[:, 0, 279].flatten(), s=1, c='red')


In [3]:
_, libs = load('Q_bu.npy', 2)
libs = libs.item()

In [ ]:
from identify_models import *
opts = dict()
opts['threshold'] = "'threshold'"
opts['row_norms'] = libs[0].row_weights
opts['gamma'] = 2
opts['verbose'] = False
opts['delta'] = 1e-8#1e-8 # apparently this reaches machine precision as the relative error
opts['char_sizes'] = libs[0].col_weights
#opts['verbose'] = True
opts1 = opts.copy()
opts1['char_sizes'] = libs[1].col_weights
opts1['row_norms'] = libs[1].row_weights

Q = libs[0].Q
#print(Q[:, 3]+Q[:, 14], np.linalg.norm(Q[:, 3]+Q[:, 14]), 
#np.linalg.norm(Q[:, 3]+Q[:, 14])/np.linalg.norm(Q[:, 0]))
#print(np.linalg.norm(Q[:, 0])/libs[0].col_weights[0])
#print(list(zip(libs[1].terms, libs[1].col_weights)))
#eqs0, lambdas0, derived_eqs0, excluded_terms = identify_equations(Q0, opts, terms0, observables, threshold=1e-5, 
#                       max_complexity=None, max_equations=999, timed=True)
#print(excluded_terms)
#eqs1, lambdas1, derived_eqs1, excluded_terms = identify_equations(Q1, opts1, terms1, observables, threshold=1e-5, 
#                       max_complexity=None, max_equations=999, timed=True, excluded_terms=excluded_terms)
#print(excluded_terms)
eqs, lambdas, derived_eqs, excluded_terms = interleave_identify([libs[0].Q, libs[1].Q], 
[opts, opts1], [libs[0].terms, libs[1].terms], threshold=2e-3)


#[0.15 s]
#Identified model: -0.9787526520510974 * rho + 0.028230207633854636 * rho * rho + -0.028610932601784253 * rho * rho[v_j * v_j] + 1.0 * rho[v_j * v_j] = 0 (order 5, residual 5.07e-05)
#[0.15 s]
#Identified model: -0.02837528778491848 * rho * rho + 1.0 * rho[v_j * v_j] = 0 (order 6, residual 1.43e-05)
#[0.26 s]
#Identified model: 0.9999199152531711 * dj rho[v_j] + 1.0 * dt rho = 0 (order 6, residual 3.51e-06)

[0.01 s]
Identified model: 0.9998092097990289 * dj rho[v_j] + 1.0 * dt rho = 0 (order 3, residual 1.70e-03)
[0.21 s]
Identified model: 0.9998215355353479 * dj rho[v_i * v_j] + 1.0 * dt rho[v_i] = 0 (order 4, residual 4.93e-04)
[0.26 s]
Identified model: 0.9998652085645199 * dj rho[v_j * v_k * v_k] + -0.0021664504398132252 * dj^2 dk rho[v_k] + -0.0021641843571197773 * dt dj^2 rho + 1.0 * dt rho[v_j * v_j] = 0 (order 5, residual 1.57e-03)
[1.84 s]
Identified model: 0.5919682680600488 * dj rho[v_j] * rho[v_k * v_k] + 1.0 * dt rho * rho[v_j * v_j] + -0.008850671228383547 * rho * rho * dj rho[v_j] + -0.0006101937910281316 * rho * rho * dt rho = 0 (order 6, residual 7.92e-04)
[10.84 s]
Identified model: 0.5926879469602502 * rho * dj rho[v_i * v_j] + 1.0 * rho * dt rho[v_i] = 0 (order 6, residual 2.27e-04)
[23.22 s]
Identified model: 0.9999093750241403 * dj rho[v_i * v_j * v_k * v_k] + -0.0028012167997592975 * dj^2 dk rho[v_i * v_k] + -0.002799905241925981 * dt dj^2 rho[v_i] + 1.0 * dt rho[v_

In [11]:
## mostly not planning to use this code

from ipynb.fs.full.sparse_reg import *
opts = dict()
opts['threshold'] = "'pareto'"
#opts['verbose'] = True
#opts['brute_force'] = False
#opts['threshold'] = "'threshold'"
#opts['n_terms'] = len(terms0)
#opts['gamma'] = 1.2
opts['char_sizes'] = srd.libs[0].col_weights
Xi, lambd, best_term, lambda1 = sparse_reg(srd.libs[0].Q, opts=opts)
print(srd.libs[0].Q.shape)

(50, 21)


In [8]:
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {srd.libs[0].terms[i]}")
print("Model residual:", lambd)
print("One-term model residual:", lambda1)
print(f"Best term (#{best_term}):", srd.libs[0].terms[best_term])

[Term 19] 1.0 * rho[v_j] * rho[v_j]
Model residual: 0.0030944418121483697
One-term model residual: 1.294046607514735e-05
Best term (#6): dt^3 rho


In [ ]:
opts['verbose'] = True
opts['char_sizes'] = srd.libs[1].col_weights
opts['avoid'] = []
opts['row_norms'] = None
Xi, lambd, best_term, lambda1 = sparse_reg(srd.libs[1].Q, opts=opts)
print(srd.libs[1].Q.shape)

In [ ]:
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {srd.libs[1].terms[i]}")
print("Model residual:", lambd)
print("One-term model residual:", lambda1)
print("Best term:", srd.libs[1].terms[best_term])

In [ ]:
### not fixed
for term, size in zip(terms1, char_sizes1):
    print(term, size)

NameError: name 'terms0' is not defined

In [4]:
continuity_terms = ['dt rho', 'dj rho[v_j]']
#continuity_terms = ['dt rho * rho', 'rho * rho * dj v_j', 'dj rho * rho * v_j']
#continuity_terms = ['dt rho', 'dj rv_j']
col_numbers = [find_term(libs[0].terms, name) for name in continuity_terms]
Xi, lambd = regress(libs[0].Q, col_numbers)
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {libs[0].terms[i]}.")# (Char. size: {srd.libs[0].col_weights[i]})")
print("Model residual:", lambd)

[Term 3] 1.0 * dt rho.
[Term 14] 0.999890075425048 * dj rho[v_j].
Model residual: 0.041745759055163256


In [7]:
#burger_terms = ['dt rho[v_i]', 'dj rho[v_i * v_j]']
burger_terms = ['rho[v_i]', 'rho[v_i * v_j * v_j]']
#burger_terms = ['rho * dt rv_i', 'dj rv_i * rv_j']
col_numbers = [find_term(libs[1].terms, name) for name in burger_terms]
Xi, lambd = regress(libs[1].Q, col_numbers)
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {libs[1].terms[i]}. (Char. size: {libs[1].col_weights[i]})")
print("Model residual:", lambd)

[Term 7] 1.0 * rho[v_i]. (Char. size: 39.88487226171494)
[Term 17] -0.6035255718187591 * rho[v_i * v_j * v_j]. (Char. size: 29913654.19628622)
Model residual: 0.27352189197837534


In [14]:
### not fixed (obviously)
#veqconst_terms = ['1', 'v_j * v_j']
veqconst_terms = ['rho', 'rho * v_j * v_j']
col_numbers = [find_term(terms0, name) for name in veqconst_terms]
Xi, lambd = regress(Q0, col_numbers)
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {terms0[i]}. (Char. size: {char_sizes0[i]})")
print("Model residual:", lambd)

[Term 16] -0.004286563115825593 * rho. (Char. size: 0.1464261231728444)
[Term 28] 1.0 * rho * v_j * v_j. (Char. size: 0.0005121399208783936)
Model residual: 0.0005697337514007149


In [ ]:
rho dt^2 v = 10^-4 dt rho * grad rho